In [ ]:
import pandas as pd
df = pd.read_csv('../data/california_housing.csv').drop(['Unnamed: 0'], axis=1)
limit = 25000

In [ ]:
# Remove outliers:
#df = df.loc[df['Population']<10000] 
#df = df.loc[df['AveOccup']<6]
#df = df.loc[df['AveBedrms']<1.5]
#df = df.loc[df['HouseAge']<50]

# # Only San Francisco :
#df = df.loc[(df['Latitude']<38.07)&(df['Latitude']>37.2)]
#df = df.loc[(df['Longitude']>-122.5)&(df['Longitude']<-121.75)]

In [ ]:
X = df.iloc[:,0:8] # the dataset
y = df.iloc[:,9] # the target variable
shap_values = df.iloc[:,[10,11,12,13,14,15,16,17]] # the SHAP values

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
regressor = GradientBoostingRegressor(random_state = 9)
regressor.fit(X, y)

In [ ]:
y_c = y>4
from sklearn.ensemble import GradientBoostingClassifier
classifier = GradientBoostingClassifier(random_state = 9)
#classifier.fit(X, y_c)

In [ ]:
from antakia.antakia import AntakIA
variables_df = pd.DataFrame(
    {'col_index': [0, 1, 2, 3, 4, 5, 6, 7],
    'descr': ['Median income', 'House age', 'Average nb rooms', 'Average nb bedrooms', 'Population', 'Average occupancy', 'Latitude', 'Longitude'],
    'type': ['float64', 'int', 'float64', 'float64', 'int', 'float64', 'float64', 'float64'],
    'unit': ['k$', 'years', 'rooms', 'rooms', 'people', 'ratio', 'degrees', 'degrees'],
    'critical': [True, False, False, False, False, False, False, False],
    'lat': [False, False, False, False, False, False, True, False],
    'lon': [False, False, False, False, False, False, False, True]},
    index=['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
)
# We call AntakIA passing the already computed SHAP values and a description of X variables :
atk = AntakIA(X.iloc[:limit], y.iloc[:limit], regressor, X_exp=shap_values)
# Here we provide the bare minimum to AntakIA : 
# atk = AntakIA(X, y, regressor)

In [ ]:
atk.start_gui()